# Setup

In [7]:
import os
import pickle
from typing import List, Dict
import uuid

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [8]:
# Setup input and output file
year = 2022
month = 2
taxi_type = "green"

input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"output/{taxi_type}/{year:04d}-{month:02d}.parquet"

In [9]:
# Setup AWS credential
os.environ["AWS_PROFILE"] = "Profile1"

# Get model from S3
RUN_ID = os.getenv("RUN_ID", "95c848791a7642ff8c26794d43e410a8")
logged_model = f"s3://taxi-mlops/1/{RUN_ID}/artifacts/model"
model = mlflow.pyfunc.load_model(logged_model)
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 95c848791a7642ff8c26794d43e410a8

# Helper function

In [10]:
def read_dataframe(filename: str) -> pd.DataFrame:
    """Read DataFrame and target column from secs to mins."""
    df = pd.read_parquet(filename)

    df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60.0)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df["ride_id"] = generate_uuids(len(df))
    return df

def prepare_dictionaries(df: pd.DataFrame) -> List[Dict]:
    """Turn DataFrame into list of dictionary."""
    # convert data type
    categorical_str = ["PULocationID", "DOLocationID"]
    df[categorical_str] = df[categorical_str].astype(str)

    # Feature engineering
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    dicts = df[categorical + numerical].to_dict(orient="records")
    return dicts

def generate_uuids(n):
    return [str(uuid.uuid4()) for _ in range(n)]

# Inference

In [11]:
def load_model(run_id):
    logged_model = f"s3://taxi-mlops/1/{run_id}/artifacts/model"
    model = mlflow.pyfunc.load_model(logged_model)
    return model

def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result["ride_id"] = df["ride_id"]
    df_result["lpep_pickup_datetime"] = df["lpep_pickup_datetime"]
    df_result["PULocationID"] = df["PULocationID"]
    df_result["DOLocationID"] = df["DOLocationID"]
    df_result["actual_duration"] = df["duration"]
    df_result["predicted_duration"] = y_pred
    df_result["diff"] = df_result["actual_duration"] - df_result["predicted_duration"]
    df_result["model_version"] = run_id
    df_result.to_parquet(output_file, index=False)

    return df_result


In [12]:
apply_model(input_file, RUN_ID, output_file)

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,1475caca-1723-436a-8f35-f3da29ca8684,2022-02-01 00:20:21,43,238,4.150000,6.517570,-2.367570,95c848791a7642ff8c26794d43e410a8
1,1a1475d2-6936-42e6-80a5-c3a6a5483d4f,2022-02-01 00:32:26,166,24,3.083333,4.099653,-1.016320,95c848791a7642ff8c26794d43e410a8
2,ebf23724-9074-4f08-bc00-1e1b3582c16a,2022-02-01 00:17:27,226,219,27.283333,24.286033,2.997300,95c848791a7642ff8c26794d43e410a8
3,e5a54d96-8449-4b39-9953-317bae6b3557,2022-02-01 00:45:37,89,83,41.650000,35.138673,6.511327,95c848791a7642ff8c26794d43e410a8
4,edebae1b-2dc4-4f9d-a7fb-4469a94f5a8c,2022-02-01 00:06:46,7,238,23.333333,24.503996,-1.170662,95c848791a7642ff8c26794d43e410a8
...,...,...,...,...,...,...,...,...
69394,4f8f6484-702b-41d3-81fc-a24382a9bd5d,2022-02-28 23:27:00,65,87,11.000000,15.106101,-4.106101,95c848791a7642ff8c26794d43e410a8
69395,cf3a2354-2c0e-4ac7-b146-b167668e4967,2022-02-28 23:59:00,97,231,11.000000,16.450340,-5.450340,95c848791a7642ff8c26794d43e410a8
69396,a82d2a55-d9b2-4377-b090-651ae18f6336,2022-02-28 23:18:00,74,116,9.000000,13.534213,-4.534213,95c848791a7642ff8c26794d43e410a8
69397,2a785868-284d-45b6-9c21-6f4f6927712a,2022-02-28 23:31:00,42,69,8.000000,10.053152,-2.053152,95c848791a7642ff8c26794d43e410a8
